In [ ]:
from requests_futures.sessions import FuturesSession
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pprint
import requests
import re

In [ ]:
with open("./all_url.txt",'r') as f:
    text = f.readlines()
    text = [i.strip() for i in text]

In [ ]:
len(text)

In [ ]:
url_list = text

In [ ]:
session = FuturesSession()

In [ ]:
def make_request(session, url):
    future = session.get(url,headers= {'User-Agent': 'Mozilla/5.0'})
    return future

In [ ]:
import time,sys

def print_progress(futures):

    check_done = lambda x: x.done()
    check_done = np.vectorize(check_done)

    #basic percentage progress
    while not check_done(futures).all():
        time.sleep(1)
        percent = check_done(futures).mean() * 100
        sys.stdout.write("\r%d%%" % percent)
        sys.stdout.flush()    
    print("\n")


In [ ]:
%%time
#create session with 16 workers
session = FuturesSession(max_workers=32)
#make all of the requests
futures =   np.array([make_request(session,url) for url in url_list]) 
print_progress(futures)
print(futures)

In [ ]:
results = [future.result().text for future in futures]

In [ ]:
# import json

# save_r = [json.dumps(i) for i in results]
# save_r = json.dumps(save_r)

# with open("save_results.json","w") as f:
#     f.write(save_r)

In [ ]:
# import json

# with open("save_results.json","r") as f:
#     result = json.load(f)

In [ ]:
test = results[0]

In [ ]:
pprint.pprint(test)

In [ ]:
ubereat_page = []

page = test

soup = BeautifulSoup(page, 'lxml')
try:
    name = soup.find("h1").text.encode().decode("unicode-escape")
except:
    name = None

try:
    address = soup.find('p').text
except:
    address = None

try:
    stars = soup.find("div", class_="header-score-details-left-score").text.strip()
except:
    stars = None

try:
    review_count = soup.find("span", itemprop="reviewCount").text.strip()
except:
    review_count = None

try:
    bookmarks = soup.find("div", class_="header-bookmark-count js-header-bookmark-count").text.strip()
except:
    bookmarks = None

try:
    district = soup.find("div", class_="header-poi-district dot-separator").text.strip()
except:
    district = None

try:
    price_range = soup.find("div", itemprop="priceRange").text.strip()
except:
    price_range = None

try:
    food_type = soup.find("div", class_="header-poi-categories dot-separator").text.strip()
except:
    food_type = None

try:
    emoji = soup.find("div", class_="header-smile-section").text.strip().split("\n\n")
except:
    emoji = None

try:
    address_ch = soup.find("section", class_="address-section").find_all("div", class_="content")[0].find("a").text.strip()
except:
    address_ch = None

try:
    address_en = soup.find("section", class_="address-section").find_all("div", class_="content")[1].find("a").text.strip()
except:
    address_en = None

try:
    transport = soup.find("section", class_="transport-section").find("div", class_="content js-text-wrapper").text.strip()
except:
    transport = None

try:
    telephone = soup.find("section", class_="telephone-section").find("div", class_="content").text.strip()
except:
    telephone = None

try:
    introduction = soup.find("section", class_="introduction-section").find("div", class_="content js-text-wrapper").text.strip().replace("\r","").replace("\n","")
except:
    introduction = None

try:
    open_hours = soup.find("div", class_="opening-hours-section js-normal-and-special-opening-hours-section").text.replace("\n","")
except:
    open_hours = None

try:
    payment = soup.find("div", class_="comma-tags").text.strip()
except:
    payment = None

try:
    review = [i.text.strip() for i in soup.find_all("div", class_="text js-text")]
except:
    review = None

ubereat_page.append({"Name" : name,
    "Delivery" : delivery,
    "Address" : address,
#     "Review_count" : review_count,
#     "Bookmarks" : bookmarks,
#     "District" : district,
#     "Price_range" : price_range,
#     "Food_type" : food_type,
#     "Emoji" : emoji,
#     "Address_ch" : address_ch,
#     "Address_en" : address_en,
#     "Transport" : transport,
#     "Telephone" : telephone,
#     "Intro" : introduction,
#     "Openhours" : open_hours,
#     "Payment" : payment,
#     "Review" : review,
    })

df = pd.DataFrame(ubereat_page)
df.to_csv("ubereats.csv", mode='w', header=True, index=False)	
print(ubereat_page)

In [ ]:
address